# RNN Building Blocks: injecting recursivity to supervised learner

In [262]:
import torch
from torch.utils.data import Dataset
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [263]:
from torch import nn


class mySeries(Dataset):
    """Series for RNN dataset."""

    def __init__(self,  length=1000, transform=None):
        self.transform = transform
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        X = idx+2
        y = idx+4

        return X,y

# mySeries dataset
trainset = mySeries()
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64)

In [264]:
class RSuperviseL(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(1, 1)

  def forward(self, x):

    x= self.fc1(x)
    return x

model = RSuperviseL()

In [265]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)
num_epochs = 1000
train_tracker, test_tracker, accuracy_tracker = [], [], []

In [266]:
for i in range(num_epochs):
    cum_loss = 0

    for batch, (X, y) in enumerate(trainloader,1):
        X = X.type(torch.float32)
        y = y.type(torch.float32)
        optimizer.zero_grad()
        output = model(X.view(-1,1))
        loss = criterion(output, y.view(-1,1))
        loss.backward()
        optimizer.step()

        cum_loss += loss.item()

    train_tracker.append(cum_loss/len(trainloader))
print(f"Epoch({i+1}/{num_epochs}) | Training loss: {cum_loss/len(trainloader)} | ")


Epoch(1000/1000) | Training loss: 0.40327949904312854 | 


In [267]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[1.0000]], requires_grad=True)
Parameter containing:
tensor([1.9918], requires_grad=True)
